# Tool Calling LLM

This notebook provides a quick overview for getting started with Tool Calling LLM for [chat models](/docs/concepts/#chat-models). For detailed documentation of all ToolCallingLLM features and configurations head to the [API reference](https://api.python.langchain.com/en/latest/llms/langchain_experimental.llms.tool_calling_llm.ToolCallingLLM.html).

Tool Calling LLM is a python mixin that lets you add [tool calling capabilities](https://python.langchain.com/v0.2/docs/concepts/#functiontool-calling) effortlessly to [`LangChain Chat Models`](https://python.langchain.com/v0.2/docs/integrations/chat/) that don't yet support tool/function calling natively. 
Simply create a new chat model class with `ToolCallingLLM` and your favorite chat model to get started.  

With `ToolCallingLLM` you also get access to [`.with_structured_output()`](https://python.langchain.com/v0.2/docs/how_to/structured_output/) which will allow you to return structured data from your model.  

At this time, `ToolCallingLLM` has been tested to work with [`ChatNVIDIA`](https://python.langchain.com/v0.2/docs/integrations/chat/nvidia_ai_endpoints/) and [`ChatLiteLLM`](https://python.langchain.com/v0.2/docs/integrations/chat/litellm/) with Ollama provider.  

The [`OllamaFunctions`](https://python.langchain.com/v0.2/docs/integrations/chat/ollama_functions/) which was the original inspiration for this effort has also been ported over to use `ToolCallingLLM`. The code for `ToolCallingLLM` was abstracted out of `OllamaFunctions` to allow it to be reused with other non tool calling Chat Models.

## Overview

### Integration details

| Class | Package | Local | Serializable | JS support | Package downloads | Package latest |
|:-----:|:-------:|:-----:|:------------:|:----------:|:-----------------:|:--------------:|
| [ToolCallingLLM](https://api.python.langchain.com/en/latest/llms/langchain_experimental.llms.tool_calling_llm.ToolCallingLLM.html) | [langchain-experimental](https://api.python.langchain.com/en/latest/openai_api_reference.html) | ✅ | ❌ | ❌ | ![PyPI - Downloads](https://img.shields.io/pypi/dm/langchain-experimental?style=flat-square&label=%20) | ![PyPI - Version](https://img.shields.io/pypi/v/langchain-experimental?style=flat-square&label=%20) |

### Model features
| [Tool calling](/docs/how_to/tool_calling/) | [Structured output](/docs/how_to/structured_output/) | JSON mode | Image input | Audio input | Video input | [Token-level streaming](/docs/how_to/chat_streaming/) | Native async | [Token usage](/docs/how_to/chat_token_usage_tracking/) | [Logprobs](/docs/how_to/logprobs/) |
| :---: | :---: | :---: | :---: |  :---: | :---: | :---: | :---: | :---: | :---: |
| ✅ | ✅ | ✅ | ✅ | ❌ | ❌ | ❌ | ✅ | ❌ | ❌ | 

## Setup

To access `ToolCallingLLM` you will need to install `langchain-experimental` integration package. You will also need to install integration packages necessary for the chat model you wish to extend using `ToolCallingLLM`.

### Credentials

Any credentials requirements for your chat model will also be applicable when extending with `ToolCallingLLM`.

### Installation

The `ToolCallingLLM` class lives in the `langchain-experimental` package:

In [ ]:
%pip install -qU langchain-experimental

## Instantiation

`ToolCallingLLM` cannot be used directly. In order to use it, you start with a chat model which doesn't already support tool calling, and extend it with with `ToolCallingLLM`. See below example of extending `ChatLiteLLM` with tool calling.

In [1]:
from typing import Any

# Example implementation using LiteLLM
from langchain_community.chat_models import ChatLiteLLM
from langchain_experimental.llms.tool_calling_llm import ToolCallingLLM


class LiteLLMFunctions(ToolCallingLLM, ChatLiteLLM):
    def __init__(self, **kwargs: Any) -> None:
        super().__init__(**kwargs)

    @property
    def _llm_type(self) -> str:
        return "litellm_functions"


llm = LiteLLMFunctions(model="ollama/phi3")

## Invocation

In [5]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
ai_msg

AIMessage(content="J'aime programmer en français.", id='run-012096e5-caa5-4dda-8863-09ffa30551d7-0')

In [6]:
ai_msg.content

"J'aime programmer en français."

## Chaining

We can [chain](/docs/how_to/sequence/) our model with a prompt template like so:

In [7]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that translates {input_language} to {output_language}.",
        ),
        ("human", "{input}"),
    ]
)

chain = prompt | llm
chain.invoke(
    {
        "input_language": "English",
        "output_language": "German",
        "input": "I love programming.",
    }
)

AIMessage(content='Es freut mich, dass Sie sich für das Programmieren interessieren! Das ist eine faszinierende Fähigkeit.', id='run-3f513ffc-b866-4caf-b700-16ff977fd26c-0')

## Tool Calling

ToolCallingLLM was designed to allow you to use tool calling features with Chat Models that don't already support that natively.

### ToolCallingLLM.bind_tools()

With `ToolCallingLLM.bind_tools`, we can easily pass in Pydantic classes, dict schemas, LangChain tools, or even functions as tools to the model. Under the hood these are converted to a tool definition schemas, which looks like:


In [8]:
from langchain_core.pydantic_v1 import BaseModel, Field


class GetWeather(BaseModel):
    """Get the current weather in a given location"""

    location: str = Field(..., description="The city and state, e.g. San Francisco, CA")


llm_with_tools = llm.bind_tools([GetWeather])

In [9]:
ai_msg = llm_with_tools.invoke(
    "what is the weather like in San Francisco",
)
ai_msg

AIMessage(content='', id='run-412dd250-1c87-4ea9-8dc1-70eb848c5899-0', tool_calls=[{'name': 'GetWeather', 'args': {'location': 'San Francisco, CA'}, 'id': 'call_fc9a5ceb5ee3483f9dff5a29013d54f9'}])

### AIMessage.tool_calls
Notice that the AIMessage has a `tool_calls` attribute. This contains in a standardized ToolCall format that is model-provider agnostic.

In [10]:
ai_msg.tool_calls

[{'name': 'GetWeather',
  'args': {'location': 'San Francisco, CA'},
  'id': 'call_fc9a5ceb5ee3483f9dff5a29013d54f9'}]

For more on binding tools and tool call outputs, head to the [tool calling](/docs/how_to/function_calling) docs.

## Structured Output

ToolCallingLLM also supports structured outputs.

With `ToolCallingLLM.with_structured_output()` you can specify a Pydantic class or a json schema to define the structure of the output you desire from the llm. 

In [11]:
from typing import Optional

from langchain_core.pydantic_v1 import BaseModel, Field


class Joke(BaseModel):
    """Joke to tell user."""

    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")
    rating: Optional[int] = Field(description="How funny the joke is, from 1 to 10")


structured_llm = llm.with_structured_output(Joke)
structured_llm.invoke("Tell me a joke about cats")

Joke(setup='A cat walks into a bar and says...', punchline="Meow! I'll have the beer, but no cheese.", rating=8)

## API reference

For detailed documentation of all ToolCallingLLM features and configurations head to the API reference: https://api.python.langchain.com/en/latest/llms/langchain_experimental.llms.tool_calling_llm.ToolCallingLLM.html